B1:

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import json

# Khởi tạo tokenizer và mô hình PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

# Hàm mã hóa văn bản dài
def encode_long_text(text, tokenizer, model, max_length=512):
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    chunk_vectors = []
    for chunk in chunks:
        inputs = tokenizer(" ".join(chunk), return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs)
        chunk_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        chunk_vectors.append(chunk_vector)
    return chunk_vectors


B2

In [ ]:
# Ví dụ danh sách các tài liệu dài
documents = ["Tài liệu dài số 1 ...", "Tài liệu dài số 2 ...", ...]

# Mã hóa và lưu trữ các vector biểu diễn
document_vectors = []
for doc in documents:
    vectors = encode_long_text(doc, tokenizer, model)
    document_vectors.append(vectors)

# Lưu trữ vector vào file để sử dụng lại sau này
with open('document_vectors.json', 'w') as f:
    json.dump(document_vectors, f)


Bước 3: Mã Hóa Câu Truy Vấn

In [ ]:
# Câu truy vấn
query = "Tôi muốn tìm hiểu về ngôn ngữ tự nhiên"

# Mã hóa câu truy vấn
query_tokens = tokenizer.tokenize(query)
query_inputs = tokenizer(" ".join(query_tokens), return_tensors="pt", padding=True, truncation=True, max_length=512)
with torch.no_grad():
    query_outputs = model(**query_inputs)
query_vector = query_outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


Bước 4: Tính Toán Độ Tương Đồng

In [ ]:
# Hàm tính độ tương đồng cosine
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Tính toán độ tương đồng giữa câu truy vấn và các đoạn của mỗi tài liệu
similarities = []
for doc_vectors in document_vectors:
    doc_similarities = [cosine_similarity(query_vector, vec) for vec in doc_vectors]
    max_similarity = max(doc_similarities)
    similarities.append(max_similarity)

# Sắp xếp các tài liệu theo độ tương đồng giảm dần
sorted_docs = sorted(zip(documents, similarities), key=lambda x: x[1], reverse=True)

# In ra các tài liệu có độ tương đồng cao nhất
for doc, similarity in sorted_docs[:10]:  # Top 10 tài liệu tương đồng nhất
    print(f"Tài liệu: {doc[:100]}... (độ tương đồng: {similarity:.4f})")
